In [1]:
# nuclio: ignore
import nuclio

In [2]:
%nuclio config kind = "job"
%nuclio config spec.image = "mlrun/ml-models"

%nuclio: setting kind to 'job'
%nuclio: setting spec.image to 'mlrun/ml-models'


In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from mlrun.execution import MLClientCtx
from mlrun.datastore import DataItem
from mlrun.artifacts import PlotArtifact, TableArtifact
from mlrun.mlutils import gcf_clear
import mlrun

from typing import List

In [5]:
pd.set_option("display.float_format", lambda x: "%.2f" % x)

def summarize(
    context: MLClientCtx,
    table: DataItem,
    label_column: str = None,
    class_labels: List[str] = [],
    plot_hist: bool = True,
    plots_dest: str = "plots",
    update_dataset = False,
) -> None:
    """Summarize a table

    :param context:         the function context
    :param table:           MLRun input pointing to pandas dataframe (csv/parquet file path)
    :param label_column:    ground truth column label
    :param class_labels:    label for each class in tables and plots
    :param plot_hist:       (True) set this to False for large tables
    :param plots_dest:      destination folder of summary plots (relative to artifact_path)
    :param update_dataset:  when the table is a registered dataset update the charts in-place 
    """
    df = table.as_df()
    header = df.columns.values
    extra_data = {}
    
    try:
        gcf_clear(plt)
        snsplt = sns.pairplot(df, hue=label_column)#, diag_kws={"bw": 1.5})
        extra_data["histograms"] = context.log_artifact(PlotArtifact("histograms",  body=plt.gcf()),
                                                        local_path=f"{plots_dest}/hist.html", db_key=False)
    except Exception as e:
        context.logger.error(f'Failed to create pairplot histograms due to: {e}')
    
    try:
        gcf_clear(plt)
        plot_cols = 3
        plot_rows = int((len(header) - 1) / plot_cols)+1
        fig, ax = plt.subplots(plot_rows, plot_cols, figsize=(15, 4))
        fig.tight_layout(pad=2.0)
        for i in range(plot_rows * plot_cols):
            if i < len(header):
                sns.violinplot(x=df[header[i]], ax=ax[int(i / plot_cols)][i % plot_cols], 
                            orient='h', width=0.7, inner="quartile")
            else:
                fig.delaxes(ax[int(i / plot_cols)][i % plot_cols])        
            i+=1
        extra_data["violin"] = context.log_artifact(PlotArtifact("violin",  body=plt.gcf(), title='Violin Plot'),
                                                    local_path=f"{plots_dest}/violin.html", db_key=False)
    except Exception as e:
        context.logger.warn(f'Failed to create violin distribution plots due to: {e}')

    if label_column: 
        labels = df.pop(label_column)
        imbtable = labels.value_counts(normalize=True).sort_index()
        try:
            gcf_clear(plt)  
            balancebar = imbtable.plot(kind='bar', title='class imbalance - labels')
            balancebar.set_xlabel('class')
            balancebar.set_ylabel("proportion of total")
            extra_data["imbalance"] = context.log_artifact(PlotArtifact("imbalance", body=plt.gcf()), 
                                                        local_path=f"{plots_dest}/imbalance.html")
        except Exception as e:
            context.logger.warn(f'Failed to create class imbalance plot due to: {e}')
        context.log_artifact(TableArtifact("imbalance-weights-vec", 
                                           df=pd.DataFrame({"weights": imbtable})),
                             local_path=f"{plots_dest}/imbalance-weights-vec.csv", db_key=False)

    tblcorr = df.corr()
    mask = np.zeros_like(tblcorr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True
    
    dfcorr = pd.DataFrame(data=tblcorr, columns=header, index=header)
    dfcorr = dfcorr[np.arange(dfcorr.shape[0])[:, None] > np.arange(dfcorr.shape[1])]
    context.log_artifact(TableArtifact("correlation-matrix", df=tblcorr, visible=True), 
                         local_path=f"{plots_dest}/correlation-matrix.csv", db_key=False)
    
    try:
        gcf_clear(plt)
        ax = plt.axes()
        sns.heatmap(tblcorr, ax=ax, mask=mask, annot=False, cmap=plt.cm.Reds)
        ax.set_title("features correlation")
        extra_data["correlation"] = context.log_artifact(PlotArtifact("correlation",  body=plt.gcf(), title='Correlation Matrix'),
                                                        local_path=f"{plots_dest}/corr.html", db_key=False)
    except Exception as e:
            context.logger.warn(f'Failed to create features correlation plot due to: {e}')
    

    gcf_clear(plt)
    if update_dataset and table.meta and table.meta.kind == 'dataset':
        from mlrun.artifacts import update_dataset_meta
        update_dataset_meta(table.meta, extra_data=extra_data)
        

In [6]:
# nuclio: end-code

### MLconfig

In [7]:
mlrun.set_environment(api_path = 'http://mlrun-api:8080',
                      artifact_path = os.path.abspath('./'))

> 2020-10-18 16:41:22,161 [warning] warning!, server (0.5.1) and client (0.5.2) ver dont match


'/User/functions/describe'

### Save

In [8]:
# create job function object from notebook code
fn = mlrun.code_to_function("describe", handler="summarize",
                                        description="describe and visualizes dataset stats",
                                        categories=["analysis"],
                                        labels = {"author": "yjb"},
                                        code_output='.')

fn.export()

> 2020-10-18 16:41:34,410 [info] function spec saved to path: function.yaml


## Tests

In [9]:
fn.apply(mlrun.platforms.auto_mount())

In [10]:
DATA_URL = 'https://s3.wasabisys.com/iguazio/data/iris/iris_dataset.csv'

task = mlrun.NewTask(name="tasks-describe", 
                     handler=summarize, 
                     inputs={"table": DATA_URL}, 
                     params={'update_dataset': True, 
                             'label_column': 'label'})

### Run Locally

In [11]:
run = mlrun.run_local(task)

> 2020-10-18 16:41:34,445 [warning] warning!, server (0.5.1) and client (0.5.2) ver dont match
> 2020-10-18 16:41:34,446 [info] starting run tasks-describe uid=082422b5bb7340208c711eb8c0bbfac5  -> http://mlrun-api:8080
> 2020-10-18 16:41:34,480 [warning] warning!, server (0.5.1) and client (0.5.2) ver dont match


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...c0bbfac5,0,Oct 18 16:41:34,completed,tasks-describe,v3io_user=idanbkind=handlerowner=idanbhost=idan-jupyter-5f8568c8b5-cpvv6,table,update_dataset=Truelabel_column=label,,histogramsviolinimbalanceimbalance-weights-veccorrelation-matrixcorrelation


to track results use .show() or .logs() or in CLI: 
!mlrun get run 082422b5bb7340208c711eb8c0bbfac5 --project default , !mlrun logs 082422b5bb7340208c711eb8c0bbfac5 --project default
> 2020-10-18 16:41:42,642 [info] run executed, status=completed


### Run Remotely

In [12]:
fn.run(task, inputs={"table": DATA_URL})

> 2020-10-18 16:41:42,656 [warning] warning!, server (0.5.1) and client (0.5.2) ver dont match
> 2020-10-18 16:41:42,657 [info] starting run tasks-describe uid=ec8eea299ec54c8ebcbd691842d3d8d0  -> http://mlrun-api:8080
> 2020-10-18 16:41:42,775 [info] Job is running in the background, pod: tasks-describe-jrdws
> 2020-10-18 16:41:54,963 [info] run executed, status=completed
final state: succeeded


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...42d3d8d0,0,Oct 18 16:41:49,completed,tasks-describe,v3io_user=idanbkind=jobowner=idanbhost=tasks-describe-jrdws,table,update_dataset=Truelabel_column=label,,histogramsviolinimbalanceimbalance-weights-veccorrelation-matrixcorrelation


to track results use .show() or .logs() or in CLI: 
!mlrun get run ec8eea299ec54c8ebcbd691842d3d8d0 --project default , !mlrun logs ec8eea299ec54c8ebcbd691842d3d8d0 --project default
> 2020-10-18 16:42:02,009 [info] run executed, status=completed
